In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

In [3]:
#### load the model file

model=load_model("model.h5")

with open ('label_gender_encode.pkl','rb') as file:
    label_gender_encode=pickle.load(file)

with open ('one_hot_geo.pkl','rb') as file:
    one_hot_geo=pickle.load(file)

with open ('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [ ]:
#### encoding the gender
input_df['Gender']=label_gender_encode.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
### encoding the geography


geo_encoded=one_hot_geo.transform(input_df[['Geography']]).toarray()
geo_enocded_df=pd.DataFrame(geo_encoded,columns=one_hot_geo.get_feature_names_out(['Geography']))
geo_enocded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
#### concating the geography columns


input_df=pd.concat([input_df.drop('Geography',axis=1),geo_enocded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
### scaling the input data

input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [11]:
### prediction 

predict=model.predict(input_scaled)
predict

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


array([[0.02640306]], dtype=float32)

In [12]:
### prediction probability


predict_prob=predict[0][0]
predict_prob

np.float32(0.02640306)

In [13]:
### chances 

if predict_prob >0.5:
    print("The person will be leaving")
else:
    print("The person will stay")

The person will stay
